# Bài tập mẫu


**Viết ứng dụng Wordcount dùng khung MapReduce:** Đếm số lần xuất hiện của mỗi từ (word) trong tệp dữ liệu đầu vào

Gợi ý:

- Xây dựng Mapper và Reducer riêng lẻ

- Với hỗ trợ của gói `Hadoop streaming` (Link:https://hadoop.apache.org/docs/r1.2.1/streaming.html#How+Streaming+Works),  Mapper và Reducer là các tệp thực thi đọc đầu vào từ luồng stdin (theo từng dòng) và ghi kết quả đầu ra cho stdout. 

- Mapper: Đọc từng dòng từ stdin và tách các từ; sau đó, mỗi từ được gán cho số đếm = 1. Lưu dữ liệu đầu ra dưới dạng <key,value> pair

- Map Output cần được sắp xếp (sort) trước khi đưa vào Reducer.

- Reducer: Khi tác vụ Reduce chạy, nó sẽ chuyển đổi các cặp khóa/giá trị đầu vào của nó thành các dòng và cung cấp các dòng này cho stdin của tiến trình (process). Trong quá trình chạy, Reducer thu thập các kết quả hướng dòng từ stdout của tiến trình, chuyển đổi từng dòng thành một <key,value> pair.


**Tạo Mapper**

In [7]:
%%writefile word_count_mapper2.py

import sys 


for line in sys.stdin:

    # remove leading and trailing whitespace 
    line = line.strip() 
    # split the line into words 
    words = line.split() 
    # increase counters 
    for word in words: 
        print('%s\t%s' % (word, 1))


Overwriting word_count_mapper2.py


**Chạy thử kiểm tra Mapper với bash script (sử dụng Cell magic %%bash)**

In [16]:
%%cmd 
cat lyric.txt | py word_count_mapper2.py  

Microsoft Windows [Version 10.0.19044.2728]
(c) Microsoft Corporation. All rights reserved.

(NLP) F:\github\BigDataWork>cat lyric.txt | py word_count_mapper2.py  


'cat' is not recognized as an internal or external command,
operable program or batch file.



(NLP) F:\github\BigDataWork>

**Tạo Reducer**

In [6]:
%%writefile word_count_reducer2.py

import sys
current_word = None
current_count = 0
word = None
for line in sys.stdin:
    # remove leading and trailing whitespaces
    line = line.strip()
    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue
    if current_word == word:
        current_count += count
    else:
        if current_word:
            print ('%s\t%s' % (current_word, current_count))
        current_count = count
        current_word = word
if current_word == word:
    print('%s\t%s' % (current_word, current_count))

Overwriting word_count_reducer2.py


# Thực thi ứng dụng trong Python:

Chạy 'MapReduce job' dùng chuỗi câu lệnh bash script:

Streaming file -> Mapping -> Sorting -> Reducing

In [8]:
%%bash
cat lyric.txt | py word_count_mapper2.py   | sort -k1,1 | py word_count_reducer2.py

'bout	1
And	1
Are	1
But	3
Even	1
Ever	2
Funny	1
Gotta	8
How	1
I	1
More	1
Sometimes	1
Tell	1
To	1
When	1
Where	6
Why	1
You've	7
a	7
all	1
and	15
are	1
ask	1
be	5
because	3
better	1
bound	3
burned	3
burns	3
by,	2
by?	2
can	1
couple	1
cry?	1
deceiving	1
desire,	3
die	3
do	1
does	1
doesn't	3
doin'?	1
doing	2
easy	1
fall	1
flame	3
flame,	3
get	18
getting	2
gonna	6
gotta	7
he's	1
heart	1
how	1
in	1
is	9
it	6
it's	2
just	6
lies?	1
love	1
make	1
me,	1
mean	3
might	1
never	1
not	1
out	1
right?	1
ruined	1
so	1
someone's	3
than	1
that	2
the	1
there	10
think	1
times	1
to	4
try	15
try,	30
turned	1
up	15
wanna	1
we	1
what	2
when	1
why	1
wonder	1
worry	1
you	3
you're	5


# Chạy Python MapReduce trong Hadoop 

**Để chạy trong Hadoop 1 ứng dụng MapReduce viết bằng Python, sử dụng gói Hadoop Streaming**.

Ngoài ra, các thư mục lưu dữ liệu đầu vào, lưu kết quả đầu ra cần được xác lập.

%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=1
hdfs dfs cp -copyFromLocal "lyric.txt" /user/input-dir

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

$HADOOP_HOME/bin/hadoop  jar $HADOOP_HOME/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files word_count_mapper2.py,word_count_reducer2.py \
    -mapper "python word_count_mapper2.py" \
    -combiner "python word_count_reducer2.py" \
    -reducer "python word_count_reducer2.py" \
    -input /user/input-dir/\
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

# Bài tập thực hành

Bài 1. Viết ứng dụng MapReduce để sắp xếp từ theo số lần xuất hiện giảm dần.


Bài 2. Viết ứng dụng MapReduce grep để tìm các dòng xuất hiện một chuỗi mẫu (pattern) của tệp đầu vào.